# VAMPNets

In [ ]:
import numpy as np
from tqdm.notebook import tqdm

import sktime
import sktime.decomposition.vampnet as vnet

import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split

In [ ]:
assert torch.cuda.is_available()
device = torch.device("cuda:0")
torch.backends.cudnn.benchmark = True
torch.set_num_threads(12)

In [ ]:
data_source = sktime.data.ellipsoids()
data = data_source.observations(100000, n_dim=5).astype(np.float32)

The dataset in two dimensions: Jump process between two metastable states where each of the states is observed in form of an ellipsoid.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.mlab as ml
from scipy.stats import multivariate_normal

x = np.linspace(-8,8,500)
y = np.linspace(-6,10,500)
X, Y = np.meshgrid(x,y)
pos = np.empty(X.shape + (2,))
pos[:, :, 0] = X
pos[:, :, 1] = Y
rv1 = multivariate_normal(data_source.state_0_mean, data_source.covariance_matrix)
rv2 = multivariate_normal(data_source.state_1_mean, data_source.covariance_matrix)

fig = plt.figure()
ax = fig.gca()

ax.contourf(X, Y, (rv1.pdf(pos) + rv2.pdf(pos)).reshape(len(x), len(y)))
ax.autoscale(False)
ax.scatter(*(data_source.observations(100).T), color='cyan', marker='x',label='samples')
plt.legend()
plt.show()

Split data into train and validation set, move the validation set into a torch tensor and onto the appropriate device.

In [ ]:
train_data, val_data = train_test_split(data, test_size=.3, shuffle=False)
val_data_tensor = torch.tensor(val_data, requires_grad=False, device=device)

The network lobe. Optionally one can use two lobes, one for the instantaneous and one for the time-shifted data.

In [ ]:
class Lobe(nn.Module):
    
    def __init__(self, fan_in, fan_out, n_hidden=5):
        super().__init__()
        layers = [nn.BatchNorm1d(fan_in), nn.Linear(fan_in, 20), nn.ELU()] \
                 + [nn.Linear(20, 20), nn.ELU()]*(n_hidden -1) \
                 + [nn.Linear(20, fan_out), nn.Softmax(1)]
        self._seq = nn.Sequential(*layers)
    
    def forward(self, inputs):
        return self._seq(inputs)

Creating an instance of the lobe.

In [ ]:
lobe = Lobe(fan_in=data.shape[1], fan_out=2, n_hidden=3).to(device=device)

The optimizer to train the brain as well as some hyperparameters.

In [ ]:
tau = 1
n_epochs = 50
batch_size = 1024
score_method = "VAMP1"

opt = torch.optim.Adam(lobe.parameters(), lr=1e-5)

In [ ]:
scores_train_x = []
scores_train = []
scores_val_x = []
scores_val = []

Now we can train the lobe using our preferred score method.

In [ ]:
step = 0
for epoch in tqdm(range(n_epochs)):
    
    ####### TRAINING #######
    lobe.train()
    for batch_0, batch_t in sktime.data.timeshifted_split(train_data, chunksize=batch_size, 
                                                          lagtime=tau, shuffle=True):
        batch_0 = torch.from_numpy(batch_0).to(device=device)
        batch_t = torch.from_numpy(batch_t).to(device=device)
        
        opt.zero_grad()
        x_0 = lobe(batch_0)
        x_t = lobe(batch_t)
        loss = vnet.loss(x_0, x_t, method=score_method)
        loss.backward()
        opt.step()
        
        scores_train_x.append(step)
        scores_train.append(loss.detach().cpu().numpy())
        step += 1
    
    ####### VALIDATION #######
    lobe.eval()
    with torch.no_grad():
        val = lobe(val_data_tensor)
        val_score = sktime.decomposition.VAMP(lagtime=tau, epsilon=1e-12)\
                        .fit(val.cpu().numpy()).fetch_model().score(score_method=score_method)

        scores_val_x.append(step)
        scores_val.append(val_score)

The scores tell us that we did not overfit and managed to improve the VAMP score a bit by the learned featurization.

In [ ]:
plt.semilogy(scores_train_x, -np.array(scores_train), label='train')
plt.semilogy(scores_val_x, scores_val, label='val')
plt.legend();

Now we can obtain a koopman model from our learnt featurization.

In [ ]:
koopman_model = vnet.VAMPNet(lagtime=tau, lobe=lobe).fit(data).fetch_model()

This model can be used to further transform the data, compute timescales and more.

In [ ]:
projection = koopman_model.transform(data)
dtraj = sktime.clustering.KmeansClustering(2).fit(projection).transform(projection)
msm = sktime.markov.msm.MaximumLikelihoodMSM().fit(dtraj, lagtime=1).fetch_model()

In [ ]:
print("estimated transition matrix", msm.transition_matrix)
print("reference transition matrix", data_source.msm.transition_matrix)

The population of states from the data should be roughly 50/50.

In [ ]:
def print_states_pie_chart():
    coors = []
    n_states = np.max(dtraj)+1

    for i in range(n_states):
        coors.append(np.sum(dtraj==i))
    total = len(dtraj)
    
    fig1, ax1 = plt.subplots()
    ax1.pie(np.array(coors), autopct='%1.2f%%', startangle=90)
    ax1.axis('equal')
    print('States population: '+str(np.array(coors)/total*100)+'%')
    plt.show()

print_states_pie_chart()

The projection can be visualized (here just the first 550 steps) also comparing to the vanilla estimator:

In [ ]:
linear_model = sktime.decomposition.VAMP(lagtime=1, dim=1).fit(data).fetch_model()

In [ ]:
plt.plot(projection[:550][:, 0], label='VAMPNet estimator');
plt.plot(linear_model.transform(data)[:550][:, 0], label='VAMP estimator', linestyle='dotted')
plt.legend();

The estimated timescales are larger then the ones we would have gotten by just using the plain data:

In [ ]:
print('VAMPNet timescale:', koopman_model.timescales()[0])
print('VAMP timescale:', linear_model.timescales()[0])

In [ ]:
print('VAMPNet score:', koopman_model.score())
print('VAMP score:', linear_model.score())